## reporjecting shape file using python

- WRF output netcdf(nc) or wrfncx.py output nc files are converted into geotiff(tiff) file with coustom projections, see the codes for nc to 
  tiff conversion.
```
from osgeo import gdal
from osgeo import osr
import numpy
import numpy.ma as ma
datafile = 'ZZZG3wrfout_psl.nc'
proj_out = osr.SpatialReference()
proj_out.SetMercator(0.0, 115.02, 0.98931892612652, 0.0, 0.0)
ds_in = gdal.Open(datafile)
#subdatasets = ds_in.GetSubDatasets()
#variables = []
#for subdataset in subdatasets:
#     variables.append(subdataset[1].split(" ")[1])
ds_lon = gdal.Open('NETCDF:"ZZZG3wrfout_psl.nc":lon')
ds_lat = gdal.Open('NETCDF:"ZZZG3wrfout_psl.nc":lat')
longs = ds_lon.GetRasterBand(1).ReadAsArray()
lats = ds_lat.GetRasterBand(1).ReadAsArray()
ds_lon = None
ds_lat = None
proj_gcp = osr.SpatialReference()
proj_gcp.ImportFromEPSG(4326)
transf = osr.CoordinateTransformation(proj_gcp, proj_out)
ul = transf.TransformPoint(float(longs[0][0]), float(lats[0][0]))
lr = transf.TransformPoint(float(longs[len(longs)-1][len(longs[0])-1]),
float(lats[len(longs)-1][len(longs[0])-1]))
ur = transf.TransformPoint(float(longs[0][len(longs[0])-1]),
float(lats[0][len(longs[0])-1]))
ll = transf.TransformPoint(float(longs[len(longs)-1][0]),
float(lats[len(longs)-1][0]))
gt0 = ul[0]
gt1 = (ur[0] - gt0) / len(longs[0])
gt2 = (lr[0] - gt0 - gt1*len(longs[0])) / len(longs)
gt3 = ul[1]
gt5 = (ll[1] - gt3) / len(longs)
gt4 = (lr[1] - gt3 - gt5*len(longs) ) / len(longs[0])
gt = (gt0,gt1,gt2,gt3,gt4,gt5)
ds_u10 = gdal.Open('NETCDF:"'+datafile+'":psl')
#i=23
#num_bands = ds_u10.RasterCount
x_size = ds_u10.RasterXSize
y_size = ds_u10.RasterYSize
ds_p_b = ds_u10.GetRasterBand(1).ReadAsArray()
driver = gdal.GetDriverByName( 'GTiff' )
ds_out = driver.Create( 'G3tas.tiff', x_size, y_size, 1, gdal.GDT_Float32 )
ds_out.SetGeoTransform( gt )
ds_out.SetProjection(proj_out.ExportToWkt())
ds_out.GetRasterBand(1).WriteArray( ds_p_b )
```
- to make a shape file to overlay on this geotiff files, shape file has to converted into the same projection as of geotiff, the code is 
  based on [this](http://www.gis.usu.edu/~chrisg/python/2009/lectures/ospy_hw2b.py) 
  and [this](http://geoinformaticstutorial.blogspot.in/2012/10/reprojecting-shapefile-with-gdalogr-and.html), the code is as follows
```python
# import modules
import ogr, osr, os, sys
# set the working directory
os.chdir('/home/swl-sacon-dst/Documents/GISE_2013/LAB/GRIBJSON_inst/wrfncxnj-0.1_r2120_old/')
# get the shapefile driver
driver = ogr.GetDriverByName('ESRI Shapefile')
# create the input SpatialReference
inSpatialRef = osr.SpatialReference()
inSpatialRef.ImportFromEPSG(4326)
outSpatialRef = osr.SpatialReference()
outSpatialRef.SetMercator(0.0, 115.02, 0.98931892612652, 0.0, 0.0)
# create the CoordinateTransformation
coordTrans = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)
# open the input data source and get the layer
inDS = driver.Open('CBE_Boundary.shp', 0)
if inDS is None:
      print 'Could not open file'
      sys.exit(1)
inLayer = inDS.GetLayer()
# create a new data source and layer
fn = 'hw2b.shp'
if os.path.exists(fn):
  driver.DeleteDataSource(fn)
outDS = driver.CreateDataSource(fn)
if outDS is None:
  print 'Could not create file'
  sys.exit(1)
outLayer = outDS.CreateLayer('hw2b', geom_type=ogr.wkbPolygon)
# get the FieldDefn for the county name field
feature = inLayer.GetFeature(0)
fieldDefn = feature.GetFieldDefnRef('id')
# add the field to the output shapefile
outLayer.CreateField(fieldDefn)
# get the FeatureDefn for the output shapefile
featureDefn = outLayer.GetLayerDefn()
# loop through the input features
inFeature = inLayer.GetNextFeature()
while inFeature:
  # get the input geometry
  geom = inFeature.GetGeometryRef()
  # reproject the geometry
  geom.Transform(coordTrans)
  # create a new feature
  outFeature = ogr.Feature(featureDefn)
  # set the geometry and attribute
  outFeature.SetGeometry(geom)
  outFeature.SetField('id', inFeature.GetField('id'))
  # add the feature to the shapefile
  outLayer.CreateFeature(outFeature)
  # destroy the features and get the next input feature
  outFeature.Destroy
  inFeature.Destroy
  inFeature = inLayer.GetNextFeature()
# close the shapefiles
inDS.Destroy()
outDS.Destroy()
# create the *.prj file
outSpatialRef.MorphToESRI()
file = open('hw2b.prj', 'w')
file.write(outSpatialRef.ExportToWkt())
file.close()
```
- this reprojection also results in UTM conversion, there by provide the shape file attributes such as area and length in meter or km.